In [1]:
import os
import numpy as np
import pandas as pd
from scipy.linalg import expm
from scipy.optimize import fsolve

## Compute variables

Read the data from the example files. Compute $Pu_0$, $\Phi_0$, $\overline{\phi}$ and $\overline{\phi_E}$.

In [2]:
spectra = pd.read_csv('spectra.csv', index_col=0)

plutonium = pd.read_csv('plutonium.csv', index_col=0)

day_0 = plutonium.index[-1]
t_0 = day_0 * 60 * 60 * 24 #convert from days to seconds

pu_0 = plutonium.loc[day_0,:].sum()

phi_E_mean = spectra.mean(axis=1)

flux_mean = phi_E_mean.sum()

phi_0 = flux_mean * t_0

## Read Cross Section Data from JANIS files

In [3]:
xs_janis = pd.read_csv('janis_xs_B_Ti.csv', index_col=0, header=[0,1])
xs_values = xs_janis.xs('sigma', level = 1, axis = 1)

### Calculate cross-sections and reaction rate

In [4]:
def calc_reaction_rate(xs_data, energy_spectrum):
    """Calculate the reaction rate from the cross sections
    
    The reaction rate is calculated by integrating the
    product of cross-section and neutron flux over energy.
    Both are available on a discrete energy grid, which
    converts the integral into a sum.
    Reaction rates for all reactions in the cross-section
    DataFrame (xs_data) are calculted. All arrays need to 
    be on the same energy grid.
    
    The JANIS cross-section data is given in units barn.
    Therefore the one-group cross-section is multiplied with
    1e-24 to transform to SI units.
    
    Parameters
    ----------
    xs_data : pd.DataFrame
        Columns contain energy dependent cross section data in 
        units of barn.
    energy_spectrum : pd.DataFrame or pd.Series
        Columns contain the energy spetrum counts.
    
    Output
    ------
    og_xs
        series, one-group cross-section for each column of xs_data
    """
    df = pd.DataFrame()
    for name, column in xs_data.items():
        prod = energy_spectrum.values * column.values
        df[name] = prod
    rate = df.sum(axis=0)
    return rate * 1e-24

In [5]:
#natural titanium composition 
n_0_Ti = np.array([0.0744,0.7372,0.0541])
#natural boron composition
n_0_B = np.array([0.2, 0.8])

In [6]:
ti_reacs = ['Ti47_MT102', 'Ti48_MT102', 'Ti49_MT102']
b_reacs = ['B10_MT107']

In [7]:
def titanium_matrix(*args):
    """Create the transition matrix for titanium"""
    t = np.array([[-args[0],        0,        0],
                  [ args[0], -args[1],        0],
                  [       0,  args[1], -args[2]]])
    return t

In [8]:
def isotopic_vector(matrix, t, xs, spectrum, n_0):
    """Calculate the vector evolution
    
    Uses the simplified burnup matrix to calculate
    the evolution of the isotopic vector.
    
    Parameters
    ----------
    matrix : callable
        Simplified burnup matrix
    t : float
        Time in seconds (time the reactor is operational)
    xs : pd.DataFrame
        Cross-section data for the reactions accounted
        for in the burnup matrix
    spectrum : np.ndarray or pd.DataFrame
        Neutron spectrum, needs to be on the same energy
        grid as the cross-sections
    n_0 : np.ndarray
        Isotopic vector of the element at t=0, commonly
        the natural isotopic composition.
        
    Returns
    -------
    iso_vec : np.ndarray
        Isotopic vector at time t
    """
    reac_rate = calc_reaction_rate(xs, spectrum)
    bu_matrix = matrix(*reac_rate.values)
    exp_matrix = expm(bu_matrix * t)
    iso_vec = np.dot(exp_matrix, n_0)
    return iso_vec

In [9]:
ti_vec = isotopic_vector(titanium_matrix,
                         t_0,
                         xs_values[ti_reacs],
                         phi_E_mean,
                         n_0_Ti
                        )

## Calculate longterm plutonium production

In [10]:
def plutonium_to_time(pu, flux_average, phi_0, pu_0):
    """Approximate time in units of plutonium
    
    With the assumption that plutonium-per-unit-fluence is constant for 
    an average batch of fuel (one simulation), the total plutonium 
    over several subsequent batches is related to the operating time
    of the reactor via a linear equation.
    
    Parameters
    ----------
    pu : float
        Plutonium density in g cm-3.
    flux_average : float
        Average flux in the reactor in s-1 cm-2.
    phi_0 : float
        Fluence of an average batch in cm-2.
    pu_0 : float
        Plutonium density of an average batch in g cm-3.
    
    Returns
    -------
    t : float
        Total irradiation time in s.
    """
    t = pu * phi_0 / pu_0 / flux_average
    return t

## Combine Steps 1 and 2

In [11]:
def ratio_plutonium_function(spectrum, phi_0, pu_0, cross_sections,
                             matrix, n_0, idx):
    """Calculate the isotopic vector as a function of plutonium
    
    Combine steps 1 and 2 of the irm analysis. First compute the 
    isotopic vector as a function of reactor operating time, then
    insert the approximation between longterm plutonium production.
    
    Parameters
    ----------
    spectrum : np.ndarray or pd.DataFrame
        Average neutron spectrum on the same energy grid
        as the cross_sections.
    phi_0 : float
        Fluence of an average batch in cm-2.
    pu_0 : float
        Plutonium density (g cm-3) in the fuel at the end of an
        average batch. 
    cross_sections : pd.DataFrame
        Cross-sections of the reactions accounted for in the 
        burnup matrix.
    matrix : callable
        The simplified burnup matrix for the isotopic vector
        of the indicator element.
    n_0 : np.ndarray
        The natural isotopic vector of the indicator element.
        
    Returns
    -------
    ratio : callable
    """
    flux_average = spectrum.sum()
    def ratio(pu):
        """Callable ratio function with plutonium as variable"""
        t = plutonium_to_time(pu, flux_average, phi_0, pu_0)
        iso_vec = isotopic_vector(matrix,
                                  t,
                                  cross_sections,
                                  spectrum,
                                  n_0
                                  )
        return iso_vec[idx[0]] / iso_vec[idx[1]]
    return ratio

In [12]:
ti_pu_func = ratio_plutonium_function(phi_E_mean, phi_0, pu_0,
                                      xs_values[ti_reacs], titanium_matrix,
                                      n_0_Ti, [1, 2])

## Solve inverse numerically to infer plutonium from ratio measurement

In [13]:
def plutonium_solver(func, ratio, guess):
    """Solve equation for plutonium given an isotopic ratio
    
    Uses scipy.optimize.fsolve to solve the equation:
    
                Ratio(Pu) - Ratio_measured = 0.
    
    Parameters
    ----------
    func : callable 
        Function relating the isotopic ratio with the total plutonium
        production.
    ratio : float
        Measured isotopic ratio.
    guess : float
        Starting guess for the solver.
        
    Returns
    -------
    pu_solve
    """
    def solve_func(pu):
        return func(pu) - ratio
    pu_solve = fsolve(solve_func, guess, full_output=True)
    return pu_solve[0]

In [14]:
ti_pu_solved = plutonium_solver(ti_pu_func, 12.92, 0.023)